In [ ]:
import os
import numpy as np
import flopy
import matplotlib.pyplot as plt

In [ ]:
# load the existing model
model_ws = './ex06-data'
s = flopy.mf6.MFSimulation().load(sim_ws=model_ws)

In [ ]:
# change work space
model_ws = './ex06-completed'
s.set_sim_path(model_ws)

In [ ]:
# get the groundwater model
gwf = s.get_model('gwt')

In [ ]:
# add head observations
head_obs = [('W1_1', 'HEAD', (0, 15, 18)), ('W1_3', 'HEAD', (2, 15, 18)), ('W1_5', 'HEAD', (4, 15, 18))]
gwobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'gwt.head.obs.csv': head_obs})

In [ ]:
# determine where the chds representing the lakes are
chd2 = gwf.get_package('LAK-1')
chd2.stress_period_data.get_data(key=0)

In [ ]:
# add chd observations
chd2obsname = 'gwt.lak.obs'
chd2.obs_filerecord.set_data([chd2obsname])
chd_obs = [('LAKE-1', 'CHD', 'lake1'), ('LAKE-2', 'CHD', 'lake2')]
chdobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'gwt.lake.obs.csv': chd_obs}, parent_file=chd2, fname=chd2obsname)

In [ ]:
# determine where the rivers are
riv = gwf.get_package('RIV-1')
riv.stress_period_data.get_data(key=0)

In [ ]:
# add river observations
rivobsname = 'gwt.riv.obs'
riv.obs_filerecord.set_data([rivobsname])
riv_obs = [('RIVER-1', 'RIV', 'seg1'), ('RIVER-2', 'RIV', 'seg2'), 
           ('RIVER-3', 'RIV', 'seg3'), ('RIVER-4', 'RIV', 'seg4')]
rivobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'gwt.riv.obs.csv': riv_obs}, parent_file=riv, fname=rivobsname)

In [ ]:
# write the model files to ex06-completed and run the simulation
s.write_simulation()
s.run_simulation()

In [ ]:
# retrieve the heads
hobj = flopy.utils.HeadFile(os.path.join(model_ws, 'gwt.hds'))
times= hobj.get_times()

In [ ]:
# create a spatial reference from the grb
grb = flopy.utils.MfGrdFile(os.path.join(model_ws, 'gwt.dis.grb'), )
sr = grb.get_spatialreference()
sr.get_extent()

In [ ]:
h = hobj.get_data(totim=times[0])
mm = flopy.plot.ModelMap(sr=sr)
mm.plot_array(h, masked_values=[1e+30])
c = mm.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d');

In [ ]:
h = hobj.get_data(totim=times[-1])
mm = flopy.plot.ModelMap(sr=sr)
mm.plot_array(h, masked_values=[1e+30])
c = mm.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d');

In [ ]:
# load and plot the gwt observations
gwobs = np.genfromtxt(os.path.join(model_ws, 'gwt.head.obs.csv'), delimiter=',', names=True)
for name in gwobs.dtype.names[1:]:
    plt.semilogx(gwobs['time'], gwobs[name], marker='.', label=name)
plt.legend()

In [ ]:
# load chd and river obs
chdobs = np.genfromtxt(os.path.join(model_ws, 'gwt.lake.obs.csv'), delimiter=',', names=True)
rivobs = np.genfromtxt(os.path.join(model_ws, 'gwt.riv.obs.csv'), delimiter=',', names=True)

In [ ]:
# plot the chd lake results
for name in chdobs.dtype.names[1:]:
    plt.semilogx(chdobs['time'], chdobs[name], marker='.', label=name)
plt.legend();

In [ ]:
# plot the river results
for name in rivobs.dtype.names[1:]:
    plt.semilogx(rivobs['time'], rivobs[name], marker='.', label=name)
plt.legend();